<a href="https://colab.research.google.com/github/DanielHevdeli/hafifot-tiug/blob/main/LLMs_as_annotators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/DanielHevdeli/hafifot-tiug.git

Cloning into 'hafifot-tiug'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 45 (delta 14), reused 20 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (45/45), 7.50 MiB | 7.07 MiB/s, done.
Resolving deltas: 100% (14/14), done.
